<a href="https://colab.research.google.com/github/s34836/EWD/blob/main/lab10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab - Classification 2

## Tasks
1. Load the `adult.csv` dataset. Inspect and explore the dataset, then prepare it for modelling.


In [49]:
# prompt: load adult.csv to dataframe

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

df = pd.read_csv('adult.csv', skipinitialspace=True)
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [50]:
df.shape

(48842, 15)

In [51]:
# fnlwgt - final weight (czyli czyli liczba ludzi o danej ce)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [52]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,48842.0,38.643585,13.710510,17.0,28.0,37.0,48.0,90.0
fnlwgt,48842.0,189664.134597,105604.025423,12285.0,117550.5,178144.5,237642.0,1490400.0
educational-num,48842.0,10.078089,2.570973,1.0,9.0,10.0,12.0,16.0
capital-gain,48842.0,1079.067626,7452.019058,0.0,0.0,0.0,0.0,99999.0
capital-loss,48842.0,87.502314,403.004552,0.0,0.0,0.0,0.0,4356.0
hours-per-week,48842.0,40.422382,12.391444,1.0,40.0,40.0,45.0,99.0


In [53]:
df.isnull().sum()

,0
age,0
workclass,0
fnlwgt,0
education,0
educational-num,0
marital-status,0
occupation,0
relationship,0
race,0
gender,0


In [54]:
sum_fnlwgt = df['fnlwgt'].sum()
sum_fnlwgt

np.int64(9263575662)

In [55]:
df = df.replace('?', pd.NA)
df = df.dropna()

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              45222 non-null  int64 
 1   workclass        45222 non-null  object
 2   fnlwgt           45222 non-null  int64 
 3   education        45222 non-null  object
 4   educational-num  45222 non-null  int64 
 5   marital-status   45222 non-null  object
 6   occupation       45222 non-null  object
 7   relationship     45222 non-null  object
 8   race             45222 non-null  object
 9   gender           45222 non-null  object
 10  capital-gain     45222 non-null  int64 
 11  capital-loss     45222 non-null  int64 
 12  hours-per-week   45222 non-null  int64 
 13  native-country   45222 non-null  object
 14  income           45222 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


In [57]:
categorical_features = ['workclass', 'education', 'marital-status', 'occupation',
                        'relationship', 'race', 'gender', 'native-country']
numerical_features = ['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']
for field in categorical_features:
    print(df.groupby(field).size())
    print("----------\n")


workclass
Federal-gov          1406
Local-gov            3100
Private             33307
Self-emp-inc         1646
Self-emp-not-inc     3796
State-gov            1946
Without-pay            21
dtype: int64
----------

education
10th             1223
11th             1619
12th              577
1st-4th           222
5th-6th           449
7th-8th           823
9th               676
Assoc-acdm       1507
Assoc-voc        1959
Bachelors        7570
Doctorate         544
HS-grad         14783
Masters          2514
Preschool          72
Prof-school       785
Some-college     9899
dtype: int64
----------

marital-status
Divorced                  6297
Married-AF-spouse           32
Married-civ-spouse       21055
Married-spouse-absent      552
Never-married            14598
Separated                 1411
Widowed                   1277
dtype: int64
----------

occupation
Adm-clerical         5540
Armed-Forces           14
Craft-repair         6020
Exec-managerial      5984
Farming-fishing      148

In [58]:
print(df[(df['relationship'] == 'Husband') & (df['gender'] == 'Female')])
print("------")
print(df[(df['relationship'] == 'Wife') & (df['gender'] == 'Male')])

       age workclass  fnlwgt education  educational-num      marital-status  \
23390   34   Private  175878   HS-grad                9  Married-civ-spouse   

      occupation relationship   race  gender  capital-gain  capital-loss  \
23390      Sales      Husband  White  Female             0             0   

       hours-per-week native-country income  
23390              40  United-States  <=50K  
------
       age workclass  fnlwgt  education  educational-num      marital-status  \
16856   29   Private  350162  Bachelors               13  Married-civ-spouse   

            occupation relationship   race gender  capital-gain  capital-loss  \
16856  Exec-managerial         Wife  White   Male             0             0   

       hours-per-week native-country income  
16856              40  United-States   >50K  


Dataset po drop NaN jest ok.
-----------
Dalej:


Compare different classification models for predicting the income group (`<=50K`/`>50K`) and select the right hyperparameters. Make sure you use the appropriate metrics to evaluate the models. You can use a validation set or cross validation (see: [`cross_validate`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html), [`cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html))

In [59]:
df['income'] = df['income'].map({'>50K': 1, '<=50K': 0})
X = df.drop('income', axis=1)
y = df['income']
X = pd.get_dummies(X, columns=categorical_features, drop_first=True)

In [47]:
X.shape

(45222, 96)

In [69]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

X_temp, X_test, y_temp, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=1, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=1, stratify=y_temp)


# 60% training
# 20% validation
# 20% test

In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score, f1_score
from sklearn.model_selection import GridSearchCV
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

models = {
    'Logistic Regression': LogisticRegression(),  # Change solver
    'Gaussian Naive Bayes': GaussianNB(),
    'Linear Discriminant Analysis': LinearDiscriminantAnalysis(),
    'Quadratic Discriminant Analysis': QuadraticDiscriminantAnalysis()  # Add regularization
}
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)

    y_val_pred = model.predict(X_val)
    y_val_proba = model.predict_proba(X_val)[:, 1]

    results[name] = {
        'Accuracy': accuracy_score(y_val, y_val_pred),
        'Precision': precision_score(y_val, y_val_pred),
        'Recall': recall_score(y_val, y_val_pred),
        'F1 Score': f1_score(y_val, y_val_pred),
        'model': model
    }
pprint(results)

{'Gaussian Naive Bayes': {'Accuracy': 0.5139856274184632,
                          'F1 Score': 0.4962181984872794,
                          'Precision': 0.3338988278840222,
                          'Recall': 0.9656556645851918,
                          'model': GaussianNB()},
 'Linear Discriminant Analysis': {'Accuracy': 0.8342730790491985,
                                  'F1 Score': 0.6832875554616522,
                                  'Precision': 0.6491368928141309,
                                  'Recall': 0.7212310437109724,
                                  'model': LinearDiscriminantAnalysis()},
 'Logistic Regression': {'Accuracy': 0.8479823106688779,
                         'F1 Score': 0.6632378153318639,
                         'Precision': 0.7354698533405758,
                         'Recall': 0.6039250669045495,
                         'model': LogisticRegression()},
 'Quadratic Discriminant Analysis': {'Accuracy': 0.40574903261470424,
                            

F1 Score': 0.6832875554616522 for Linear Discriminant Analysis - THE BEST

In [84]:
pprint(models['Linear Discriminant Analysis'].get_params())

{'covariance_estimator': None,
 'n_components': None,
 'priors': None,
 'shrinkage': None,
 'solver': 'svd',
 'store_covariance': False,
 'tol': 0.0001}


In [85]:
# solver{‘svd’, ‘lsqr’, ‘eigen’}
# shrinkage‘auto’ or float, default=None
# priors array-like of shape (n_classes,), default=None
#param_grid = {
# 'solver': ['svd', 'lsqr'],
#'shrinkage': [None, 'auto']
#}

param_grid = {
    'solver': ['svd', 'lsqr', 'eigen'],
    'shrinkage': [None, 'auto', 0.1, 0.3, 0.5, 0.7, 0.9],
    'tol': [0.0001, 0.001, 0.01],
    'store_covariance': [True, False],
    'n_components': [None, 1, 2, 3, 5]
}

grid = GridSearchCV(
    LinearDiscriminantAnalysis(),
    param_grid,
    scoring='f1',
    cv=3
)
grid.fit(X_train, y_train)

print("\nBest parameters:", grid.best_params_)

# Evaluate on test set
best_model = grid.best_estimator_
y_test_pred = best_model.predict(X_test)
print(f"F1 Score: {f1_score(y_test, y_test_pred)}")


Best parameters: {'n_components': None, 'shrinkage': 0.7, 'solver': 'lsqr', 'store_covariance': True, 'tol': 0.0001}
F1 Score: 0.6570527264533573
